# PLAYGROUND for AgentGENius

In [1]:
from datetime import date

import nest_asyncio
from dotenv import load_dotenv
from pydantic_ai import RunContext, Tool

from agentgenius.agents import AgentStore, BaseAgent
from agentgenius.config import config
from agentgenius.core import AgentGENius
from agentgenius.tools import ToolSet

nest_asyncio.apply()
load_dotenv()

True

## Testing AgentStore 

In [2]:
ags = AgentStore(config.agents_path).load_agents()
# ags.add(AgentGENius.from_file(config.agents_path / "assistant.json"))
a = ags.list()[0]
print(a)
if agent := ags[a]:
    print(agent.run_sync("write hello world").data)

tool_agent
```python
def write_hello_world():
    return "Hello, World!"

# Execute the function and display the result
result = write_hello_world()
result
```


## Using tools

In [3]:
def encode_rot13(input_string):
    """Encode a string using ROT13."""
    import codecs

    return codecs.encode(input_string, "rot_13")


# check signature if function is plain
a = encode_rot13
import inspect

signature = inspect.signature(a)
signature.parameters.keys()
not any("RunContext" in str(signature.parameters[t].annotation) for t in signature.parameters.keys())

True

In [4]:
agent = AgentGENius(name="rot13encoder", model="openai:gpt-4o-mini", system_prompt="you write everything in rot13")

# add tool to toolset manually
agent.toolset.add(encode_rot13)


# or use decorator
@agent.tool_plain
def ask_user_tool(question: str) -> str:
    """Ask the user a question"""
    return input(question + " ")


print(agent.toolset, "\n")
result = agent.run_sync("tell me a haiku about ciphers")
print(result.data, "\n")
print(encode_rot13(result.data))

ToolSet(['ask_agent', 'encode_rot13']) 

Juvfcref bs frpergf,  
Uvqqra va gur pbqrq yvarf,  
Gehgu hairvyrq va gvzr.   

Whispers of secrets,  
Hidden in the coded lines,  
Truth unveiled in time.  


## Testing ToolSet

In [5]:
def test_tool(value: int) -> str:
    return str(value)


tools = ToolSet(test_tool)

tool = tools.get("test_tool")
assert tool(10) == "10"

assert "test_tool2" not in tools
assert "test_tool" in tools
assert test_tool in tools

tools.remove("test_tool")
tool = tools.get("test_tool")
assert tool is None

## Testing Serialization

In [11]:
def ask_user_tool(ctx: RunContext[str], question: str) -> str:
    """Ask the user a question"""
    return input(question + " ")


tools = ToolSet(ask_user_tool)

agent = AgentGENius(
    name="assistant", model="openai:gpt-4o-mini", system_prompt="You are a helpful assistant.", toolset=tools
)


# the date will be injected into the system prompt, but only for the first agent
@agent.system_prompt
def inject_date() -> str:
    return f"The date is {date.today()}."


print(f"{agent.to_json()=}")

j = agent.to_json()
agent2 = BaseAgent.from_json(j)


print(f"{agent2.to_json()=}")
# Agents toolsets aren't the same, it's not working in the notebook for some reason

print(agent.run_sync("What is the current date?").data)
print(agent2.run_sync("What is the current date?").data)

agent.to_json()='{"name":"assistant","model":"openai:gpt-4o-mini","system_prompt":"You are a helpful assistant.","toolset":["ask_user_tool","ask_agent"]}'
agent2.to_json()='{"name":"assistant","model":"openai:gpt-4o-mini","system_prompt":"You are a helpful assistant.","toolset":["ask_agent"]}'
The current date is December 30, 2024.
I can't provide the current date directly. However, you can easily find out the current date by checking your device's calendar or doing a quick online search.
